The ENTSO-E (European Network of Transmission System Operators for Electricity) Transparency Platform, launched in 2015, serves as Europe's central hub for electricity market data. Created to fulfill EU Regulation 543/2013, it collects and publishes data from over 42 transmission system operators across Europe. The platform transformed what was once a fragmented landscape of national data sources into a unified repository, providing crucial information about power generation, consumption, and cross-border flows. For market participants and analysts, this data source is invaluable as it offers insights into the fundamental drivers of electricity prices, including generation mix, grid constraints, and demand patterns.

![Example Image](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRv8tncnUyen1EF-uNSqu9D2iw7FSClbohI4KQBGH2XPhI_7ve-W9BXXI5-vluG2t9FCLQ&usqp=CAU)

We applied for an API-Key and programmatically extract relevant information in the following.

In [ ]:
# TEST
from entsoe import EntsoeRawClient
import pandas as pd
from dotenv import load_dotenv 

load_dotenv()
ENTSOE_API_KEY = os.getenv('ENTSOE_API_KEY')
print(ENTSOE_API_KEY)



5e9e89b6-55b4-4b29-bfa7-6cfbce2478cc
<?xml version="1.0" encoding="utf-8"?>
<Publication_MarketDocument xmlns="urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:3">
  <mRID>1beb2403e9bc4f3f81754b7479686805</mRID>
  <revisionNumber>1</revisionNumber>
  <type>A44</type>
  <sender_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</sender_MarketParticipant.mRID>
  <sender_MarketParticipant.marketRole.type>A32</sender_MarketParticipant.marketRole.type>
  <receiver_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</receiver_MarketParticipant.mRID>
  <receiver_MarketParticipant.marketRole.type>A33</receiver_MarketParticipant.marketRole.type>
  <createdDateTime>2024-11-04T11:22:42Z</createdDateTime>
  <period.timeInterval>
    <start>2017-11-30T23:00Z</start>
    <end>2017-12-31T23:00Z</end>
  </period.timeInterval>
      <TimeSeries>
        <mRID>1</mRID>
        <auction.type>A01</auction.type>
        <businessType>A62</businessType>
        <in_Domain.mRID codingSche

In [ ]:
# XML client

client = EntsoeRawClient(api_key=ENTSOE_API_KEY)

start = pd.Timestamp('20171201', tz='Europe/Brussels')
end = pd.Timestamp('20180101', tz='Europe/Brussels')
country_code = 'BE'  # Belgium
country_code_from = 'FR'  # France
country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = 'A01'
process_type = 'A51'

xml_string = client.query_day_ahead_prices(country_code, start, end)
print(xml_string)

In [ ]:
# Pandas client

client = EntsoePandasClient(api_key=ENTSOE_API_KEY)

start = pd.Timestamp('20171201', tz='Europe/Brussels')
end = pd.Timestamp('20180101', tz='Europe/Brussels')
country_code = 'BE'  # Belgium
country_code_from = 'FR'  # France
country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = 'A01'
process_type = 'A51'

df = client.query_day_ahead_prices(country_code, start, end)
print(df)

2017-12-01 00:00:00+01:00    61.81
2017-12-01 01:00:00+01:00    49.34
2017-12-01 02:00:00+01:00    47.53
2017-12-01 03:00:00+01:00    49.14
2017-12-01 04:00:00+01:00    44.96
                             ...  
2017-12-31 20:00:00+01:00    19.69
2017-12-31 21:00:00+01:00    17.85
2017-12-31 22:00:00+01:00    18.94
2017-12-31 23:00:00+01:00    14.04
2018-01-01 00:00:00+01:00    14.16
Freq: 60min, Length: 745, dtype: float64


In [26]:
# API outages

def get_planned_outages(start_date, end_date, country_code='DE_LU', ENTSOE_API_KEY=None):
    """
    Fetch planned outages data from ENTSO-E API using EntsoePandasClient.
    
    Args:
        start_date (datetime): Start date for the query
        end_date (datetime): End date for the query
        country_code (str): Country code (default: 'DE_LU' for Germany-Luxembourg)
    
    Returns:
        pandas.DataFrame or None: Outages data if successful, None if failed
    """

    if not ENTSOE_API_KEY:
        raise ValueError("ENTSOE_API_KEY not found")

    # Initialize client with API key
    client = EntsoePandasClient(api_key=ENTSOE_API_KEY)
    
    try:
        # Convert dates to timezone-aware timestamps using Brussels timezone
        # as shown in the documentation example
        start = pd.Timestamp(start_date, tz='Europe/Brussels')
        end = pd.Timestamp(end_date, tz='Europe/Brussels')
        
        # Ensure the dates are within valid range
        if end - start > pd.Timedelta(days=365):
            raise ValueError("Date range cannot exceed 1 year")
        
        # Fetch outages data using the documented method
        outages = client.query_unavailability_of_production_units(
            country_code=country_code,
            start=start,
            end=end,
            docstatus=None  # None to get all statuses
        )
        
        if outages is not None and not outages.empty:
            # Reset index to make the datetime a column
            outages = outages.reset_index()
            
            # Add duration calculation
            if 'start_date' in outages.columns and 'end_date' in outages.columns:
                outages['duration_hours'] = (outages['end_date'] - outages['start_date']).dt.total_seconds() / 3600
            
            return outages
        
        return None

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        print("\nDebug information:")
        print(f"API Key length: {len(ENTSOE_API_KEY) if ENTSOE_API_KEY else 'No API key found'}")
        print(f"Start date: {start}")
        print(f"End date: {end}")
        print(f"Country code: {country_code}")
        return None

def display_outages_summary(outages_df):
    """
    Display a summary of the outages data.
    
    Args:
        outages_df (pandas.DataFrame): The outages data
    """
    if outages_df is not None and not outages_df.empty:
        print("\nDataset Summary:")
        print(f"Total number of outages: {len(outages_df)}")
        print("\nColumns in dataset:")
        print(outages_df.columns.tolist())
        
        print("\nDate range:")
        if 'start_date' in outages_df.columns:
            print(f"First outage: {outages_df['start_date'].min()}")
            print(f"Last outage: {outages_df['end_date'].max()}")
        
        if 'duration_hours' in outages_df.columns:
            print("\nDuration statistics (hours):")
            print(outages_df['duration_hours'].describe())
        
        print("\nFirst few rows:")
        print(outages_df.head())
        
        # Option to save to CSV
        save = input("\nWould you like to save the data to CSV? (y/n): ")
        if save.lower() == 'y':
            filename = f"outages_{country_code}_{start_date.strftime('%Y%m%d')}_{end_date.strftime('%Y%m%d')}.csv"
            outages_df.to_csv(filename, index=False)
            print(f"Data saved to {filename}")
    else:
        print("No data available to display")


start = datetime(2024, 1, 1)
end = datetime(2024, 1, 2)  # Just one month to test

# Note: Using DE_LU instead of DE as per the documentation's validity table
outages_df = get_planned_outages(start, end, country_code='DE_LU', ENTSOE_API_KEY=ENTSOE_API_KEY)
# display_outages_summary(outages_df)

In [27]:
outages_df

,created_doc_time,avail_qty,biddingzone_domain,businesstype,curvetype,docstatus,end,mrid,nominal_power,plant_type,production_resource_id,production_resource_location,production_resource_name,production_resource_psr_name,pstn,qty_uom,resolution,revision,start
0,2023-12-26 15:13:21+01:00,345,DE_LU,Planned maintenance,A03,Cancelled,2024-01-01 01:00:00+01:00,ZnNdUvXbhQ9HwI52BnDY7g,400.0,Wind Offshore,11WD2OGTI000264J,Nordsee AWZ,Global Tech I,,1,MAW,PT15M,2,2023-12-31 20:00:00+01:00
1,2023-12-27 21:41:20+01:00,327,DE_LU,Planned maintenance,A03,Cancelled,2024-01-01 20:45:00+01:00,vodq2qPhTbblSn_juqgH8A,400.0,Wind Offshore,11WD2OGTI000264J,Nordsee AWZ,Global Tech I,,1,MAW,PT15M,2,2024-01-01 01:00:00+01:00
2,2023-12-27 21:41:33+01:00,327,DE_LU,Planned maintenance,A03,Cancelled,2024-01-01 01:00:00+01:00,nN5iLbiIaQaHhwfXHCrUjg,400.0,Wind Offshore,11WD2OGTI000264J,Nordsee AWZ,Global Tech I,,1,MAW,PT15M,2,2023-12-31 20:45:00+01:00
3,2023-12-27 21:43:47+01:00,332,DE_LU,Planned maintenance,A03,Cancelled,2024-01-02 00:00:00+01:00,0HcvCbmriuTnKKeZ9xZZAQ,400.0,Wind Offshore,11WD2OGTI000264J,Nordsee AWZ,Global Tech I,,1,MAW,PT15M,2,2024-01-01 20:45:00+01:00
4,2023-12-28 01:35:49+01:00,277,DE_LU,Planned maintenance,A03,Cancelled,2024-01-02 00:00:00+01:00,2SO6vs5CIWsR_YiINtCURg,400.0,Wind Offshore,11WD2OGTI000264J,Nordsee AWZ,Global Tech I,,1,MAW,PT15M,2,2024-01-01 01:00:00+01:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2024-01-02 00:23:04+01:00,281.5,DE_LU,Planned maintenance,A03,None,2024-01-01 18:30:00+01:00,k2rFdAcw49gkWVXEaX_OIQ,400.0,Wind Offshore,11WD2OGTI000264J,Nordsee AWZ,Global Tech I,,1,MAW,PT15M,1,2024-01-01 17:00:00+01:00
75,2024-01-02 00:23:14+01:00,285,DE_LU,Planned maintenance,A03,Cancelled,2024-01-02 01:00:00+01:00,YElVGHvG_1PzXg5OYmeB0Q,400.0,Wind Offshore,11WD2OGTI000264J,Nordsee AWZ,Global Tech I,,1,MAW,PT15M,2,2024-01-01 19:15:00+01:00
76,2024-01-02 00:23:24+01:00,284.6,DE_LU,Planned maintenance,A03,None,2024-01-02 01:00:00+01:00,czyUxZvdqbsHP7VhSOFcew,400.0,Wind Offshore,11WD2OGTI000264J,Nordsee AWZ,Global Tech I,,1,MAW,PT15M,1,2024-01-01 22:30:00+01:00
77,2024-01-03 09:08:39+01:00,0,DE_LU,Planned maintenance,A03,None,2024-01-04 08:00:00+01:00,KomCYgInxjP18Nki-ZvybQ,220.0,Hydro Pumped Storage,11WD2ERZH0002682,intra_zonal,PSW Erzhausen,,1,MAW,PT15M,2,2024-01-01 00:00:00+01:00


In [ ]:
# Scraping outages
import requests
import pandas as pd
from bs4 import BeautifulSoup

def scrape_power_plant_maintenance():
    # This is a conceptual example - actual implementation depends on specific website
    urls = [
        'https://transparency.entsoe.eu/',
        'https://www.eex-transparency.com/'
    ]
    
    maintenance_data = []
    
    for url in urls:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Parsing logic would depend on the specific website structure
            # This is just a placeholder
            maintenance_rows = soup.find_all('tr', class_='maintenance-row')
            
            for row in maintenance_rows:
                maintenance_entry = {
                    'plant_type': row.find('td', class_='plant-type').text,
                    'start_date': row.find('td', class_='start-date').text,
                    'end_date': row.find('td', class_='end-date').text,
                    'capacity': row.find('td', class_='capacity').text
                }
                maintenance_data.append(maintenance_entry)
        
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    
    return pd.DataFrame(maintenance_data)

In [ ]:
# API next day prediction

from entsoe import EntsoePanelClient
import pandas as pd
from datetime import datetime, timedelta

def get_market_expectations(start_date, end_date):
    client = EntsoePanelClient(api_key='YOUR_API_KEY')
    
    try:
        # Fetch day-ahead price forecasts
        forecasts = client.get_day_ahead_prices(
            country_code='DE',
            start=start_date,
            end=end_date
        )
        
        # Get actual prices for comparison
        actual_prices = client.get_day_ahead_prices(
            country_code='DE',
            start=start_date + timedelta(days=1),
            end=end_date + timedelta(days=1)
        )
        
        # Combine into DataFrame
        df = pd.DataFrame({
            'forecast': forecasts,
            'actual': actual_prices
        })
        
        # Calculate forecast error
        df['forecast_error'] = df['actual'] - df['forecast']
        
        return df
    
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None